In [34]:
from Bio import SeqIO
import sys
import os
import pandas as pd
import numpy as np
import MySQLdb
from collections import Counter 
import re
import itertools
import MySQLdb

connection = MySQLdb.connect(
                host = '127.0.0.1',
                user = 'root',
                passwd = 'Pangenome19')

cursor = connection.cursor()
connection.autocommit(True)
cursor.execute("USE OGroups19")


main_dir="/Users/sigalova/Desktop/Chlamydia_pangenome/"

# Genome table

In [59]:
# prepare

cursor.execute('DROP TABLE IF EXISTS genomes;')
cursor.execute("""CREATE TABLE genomes (
genome_id VARCHAR(10) NOT NULL,
PRIMARY KEY(genome_id),
full_name TEXT,
gi INT NOT NULL,
ref VARCHAR(20) NOT NULL, 
strain TEXT,
species TEXT,
DNAlength INT DEFAULT NULL,
serovar TEXT,
specificity TEXT,
cluster TEXT,
submitter TEXT,
year_sequenced TEXT,
collected_by TEXT,
year_collected TEXT,
anomality TEXT,
comment TEXT,
publication TEXT,
contigs INT);
""")

sql_gen=""" INSERT INTO genomes (genome_id, full_name, gi, ref, strain, species, DNAlength, 
        serovar, specificity, cluster, submitter, year_sequenced, collected_by, year_collected , 
        anomality, comment, publication, contigs) 
        VALUES (%s, %s, %s, %s,%s,%s, %s, %s, %s,%s,%s, %s, %s, %s,%s,%s,%s, %s);"""



In [58]:
# Genome table
genomes=pd.read_csv(main_dir+"info/genomes_info_jun_2016.csv")

# Remove duplicate genomes and bad assemblies
genomes=pd.read_csv(main_dir+"info/genomes_info_jun_2016.csv")
genomes=genomes[genomes["specificity"]!='recombinant'].replace(np.nan,'', regex=True)
filtered=['C091','C086','C112','C113','C017','C135','C131'] # these genomes are either duplicates (strain or whole genome) or of bad assembly
genomes=genomes[~genomes["genome_id"].isin(filtered)]
genomes.index=genomes["genome_id"]

# Lists of genomes with outgroup, without outgroup, without draft genomes
genome_ids_withOutgroup=genomes["genome_id"].tolist() 
genome_ids_withoutOutgroup=[x for x in genome_ids_withOutgroup if x!="W001"] 
genome_ids_nodraft=genomes[(genomes["contigs"]==1) & (genomes["genome_id"]!="W001")]["genome_id"].tolist()    # without outgroup, 

In [60]:
# Fill database 

for i in range(genomes.shape[0]):
    cursor.execute(sql_gen,tuple(genomes.iloc[i,]))